# 

This notebook creates dummy data for an example GWAS analysis. First, let's load in the needed libraries.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.stats import norm

Next, set chromosome lengths per the hg38 reference assembly.

In [2]:
chromosome_lengths = {
    '1': 248956422, '2': 242193529, '3': 198295559, '4': 190214555, '5': 181538259,
    '6': 170805979, '7': 159345973, '8': 145138636, '9': 138394717, '10': 133797422,
    '11': 135086622, '12': 133275309, '13': 114364328, '14': 107043718, '15': 101991189,
    '16': 90338345, '17': 83257441, '18': 80373285, '19': 58617616, '20': 64444167,
    '21': 46709983, '22': 50818468, 'X': 156040895
}

total_length = sum(chromosome_lengths.values())

Now a function to simulate genotypes based on minor allele frequency.

In [3]:
def simulate_genotype(maf, n_samples):
    return np.random.choice([0, 1, 2], size=n_samples, p=[(1-maf)**2, 2*maf*(1-maf), maf**2])

Let's set some parameters for our simulated dataset.

In [4]:
# Parameters
n_samples = 50000
n_loci = 10000

Now calculate the number of loci per chromosome.

In [5]:
chromosome_loci_counts = {chrom: int(n_loci * length / total_length) for chrom, length in chromosome_lengths.items()}

In [6]:
chromosome_loci_counts

{'1': 821,
 '2': 799,
 '3': 654,
 '4': 627,
 '5': 598,
 '6': 563,
 '7': 525,
 '8': 478,
 '9': 456,
 '10': 441,
 '11': 445,
 '12': 439,
 '13': 377,
 '14': 353,
 '15': 336,
 '16': 298,
 '17': 274,
 '18': 265,
 '19': 193,
 '20': 212,
 '21': 154,
 '22': 167,
 'X': 514}

Create the info columns for the PED file. First, let's create a balanced set of phenotypes.

In [7]:
phenotypes = np.array([0] * (n_samples // 2) + [1] * (n_samples // 2))
np.random.shuffle(phenotypes)

In [8]:
np.unique(phenotypes, return_counts=True)

(array([0, 1]), array([25000, 25000]))

In [9]:
PED_info_columns = pd.DataFrame({
    'FID': [0] * n_samples,
    'IID': [f'IND{i}' for i in range(n_samples)],
    'Father_ID': [0] * n_samples,
    'Mother_ID': [0] * n_samples,
    'Sex': [0] * n_samples,
    'PHENOTYPE': phenotypes
})

In [10]:
PED_info_columns.head(5)

,FID,IID,Father_ID,Mother_ID,Sex,PHENOTYPE
0,0,IND0,0,0,0,1
1,0,IND1,0,0,0,1
2,0,IND2,0,0,0,0
3,0,IND3,0,0,0,0
4,0,IND4,0,0,0,0


Create lists to hold the generated data.

In [11]:
genotype_columns = []
variant_info = []

Now to generate genotypes per variant.

In [12]:
for chrom, n_variants in chromosome_loci_counts.items():
    chrom_length = chromosome_lengths.get(chrom)
    
    for i in range(n_variants):
        snp_id = f"rs{chrom}_{i+1}"
        maf = np.random.uniform(0.01, 0.5)  # Random MAF between 0.01 and 0.5
        genotypes = simulate_genotype(maf, n_samples)
        
        # Append genotype data
        genotype_columns.append(pd.Series(genotypes, name=snp_id))

        # Variant position
        position = np.random.randint(1, chrom_length + 1)
        
        # Store variant information
        variant_info.append([chrom, snp_id, 0, position])

In [13]:
genotype_data = pd.concat(genotype_columns, axis = 1)
PED = pd.concat([PED_info_columns, genotype_data], axis = 1)

In [14]:
PED.head(10)

,FID,IID,Father_ID,Mother_ID,Sex,PHENOTYPE,rs1_1,rs1_2,rs1_3,rs1_4,...,rsX_505,rsX_506,rsX_507,rsX_508,rsX_509,rsX_510,rsX_511,rsX_512,rsX_513,rsX_514
0,0,IND0,0,0,0,1,1,0,0,0,...,2,0,1,0,1,0,0,1,0,0
1,0,IND1,0,0,0,1,0,0,0,0,...,1,1,0,0,0,1,0,0,0,0
2,0,IND2,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
3,0,IND3,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,1,0,0,0
4,0,IND4,0,0,0,0,2,0,1,0,...,0,1,0,0,0,0,0,2,2,0
5,0,IND5,0,0,0,1,1,0,1,0,...,1,0,0,0,0,1,0,0,0,0
6,0,IND6,0,0,0,1,0,1,2,0,...,0,2,1,0,0,0,0,2,0,0
7,0,IND7,0,0,0,0,2,0,1,0,...,0,0,0,0,0,0,1,1,1,0
8,0,IND8,0,0,0,1,0,0,2,0,...,0,0,0,0,0,0,0,1,1,1
9,0,IND9,0,0,0,1,0,0,1,0,...,0,0,1,0,0,0,2,2,0,2


In [15]:
MAP = pd.DataFrame(variant_info, columns = ['chr','rsID','cM','pos'])

In [16]:
MAP.head()

,chr,rsID,cM,pos
0,1,rs1_1,0,200826415
1,1,rs1_2,0,4027976
2,1,rs1_3,0,222620360
3,1,rs1_4,0,5556784
4,1,rs1_5,0,176260783


In [17]:
def calculate_GWAS_summary_stats_from_PED(PED_file):
    # Read PED file
    df = PED_file
 
    # Extract phenotype and genotypes
    phenotypes = df.iloc[:, 5].astype(int)
    genotypes = df.iloc[:, 6:].astype(int)
    
    # Initialize lists to store results
    snp_ids = genotypes.columns.tolist()
    allele_freqs = []
    betas = []
    odds_ratios = []
    p_values = []

    # Loop through each SNP
    for snp in snp_ids:
        snp_genotypes = genotypes[snp]

        # Calculate allele frequencies
        allele_freq = snp_genotypes.mean() / 2
        allele_freqs.append(allele_freq)
        
        # Logistic regression for association testing
        X = sm.add_constant(snp_genotypes)
        model = sm.Logit(phenotypes, X)
        result = model.fit(disp=0)
        
        # Get beta, odds ratio, and p-value
        beta = result.params[snp]
        odds_ratio = np.exp(beta)
        p_value = result.pvalues[snp]
        
        betas.append(beta)
        odds_ratios.append(odds_ratio)
        p_values.append(p_value)

    # Create summary statistics dataframe
    summary_stats = pd.DataFrame({
        'SNP': snp_ids,
        'Allele_Frequency': allele_freqs,
        'Beta': betas,
        'Odds_Ratio': odds_ratios,
        'P_Value': p_values
    })

    return summary_stats

In [18]:
summary_stats = calculate_GWAS_summary_stats_from_PED(PED)

In [19]:
summary_stats.head(10)

,SNP,Allele_Frequency,Beta,Odds_Ratio,P_Value
0,rs1_1,0.42803,0.007762,1.007793,0.543708
1,rs1_2,0.06738,0.002549,1.002552,0.919577
2,rs1_3,0.18924,-0.008810,0.991229,0.584180
3,rs1_4,0.05474,-0.024823,0.975482,0.372813
4,rs1_5,0.12379,-0.013784,0.986310,0.472171
5,rs1_6,0.22503,-0.001488,0.998513,0.921653
6,rs1_7,0.40321,-0.004071,0.995938,0.752156
7,rs1_8,0.25622,-0.018303,0.981864,0.207007
8,rs1_9,0.32605,-0.014779,0.985330,0.272441
9,rs1_10,0.40165,0.006437,1.006458,0.618614


In [20]:
summary_stats[summary_stats['P_Value'] < 0.001]

,SNP,Allele_Frequency,Beta,Odds_Ratio,P_Value
469,rs1_470,0.28793,0.047525,1.048673,0.000671
1169,rs2_349,0.45021,0.046613,1.047717,0.000240
1836,rs3_217,0.42116,-0.048621,0.952542,0.000141
2656,rs4_383,0.24153,0.048885,1.050099,0.000976
2711,rs4_438,0.34079,-0.045140,0.955864,0.000719
2979,rs5_79,0.12279,0.067004,1.069300,0.000508
3147,rs5_247,0.47287,0.044800,1.045818,0.000403
3200,rs5_300,0.26862,0.052631,1.054041,0.000230
3577,rs6_79,0.29646,0.051740,1.053102,0.000186
6828,rs12_422,0.13725,-0.061698,0.940166,0.000795


There are likely a few loci that approach significance but none that reach a genome-wide threshold. Let's manually add a few. We will write a function that creates a genotype distribution based on specified genotype frequencies.

In [21]:
def generate_genotypes_for_significant_gwas(phenotypes, controls_genotype_frequencies, cases_genotype_frequencies):
    n_samples = len(phenotypes)
    n_cases = np.sum(phenotypes)
    n_controls = n_samples - n_cases
    
    # Generate control genotypes
    control_genotypes = np.random.choice(
        [0, 1, 2], size=n_controls, p=controls_genotype_frequencies)
    
    # Generate case genotypes
    case_genotypes = np.random.choice(
        [0, 1, 2], size=n_cases, p=cases_genotype_frequencies)
    
    # Combine genotypes
    genotypes = np.zeros(n_samples, dtype=int)
    genotypes[phenotypes == 0] = control_genotypes
    genotypes[phenotypes == 1] = case_genotypes
    
    return genotypes

In [22]:
sig_variant_1 = generate_genotypes_for_significant_gwas(PED['PHENOTYPE'], [0.40, 0.38, 0.22], [0.35, 0.42, 0.23])
sig_variant_2 = generate_genotypes_for_significant_gwas(PED['PHENOTYPE'], [0.40, 0.38, 0.22], [0.35, 0.42, 0.23])
sig_variant_3 = generate_genotypes_for_significant_gwas(PED['PHENOTYPE'], [0.40, 0.38, 0.22], [0.35, 0.42, 0.23])
sig_variant_4 = generate_genotypes_for_significant_gwas(PED['PHENOTYPE'], [0.40, 0.38, 0.22], [0.35, 0.42, 0.23])
sig_variant_5 = generate_genotypes_for_significant_gwas(PED['PHENOTYPE'], [0.40, 0.37, 0.23], [0.35, 0.42, 0.23])

Add these variants to the PED and MAP files. We'll add variants to chromosomes 2, 3, 4, 16, and 17. 

In [23]:
PED['rs848293'] = sig_variant_1
PED['rs10779987'] = sig_variant_2
PED['rs62340585'] = sig_variant_3
PED['rs11865086'] = sig_variant_4
PED['rs4792891'] = sig_variant_5

In [24]:
new_MAP_rows = pd.DataFrame({
    'chr': [2,3,4,16,17],
    'rsID': ['rs848293','rs10779987','rs62340585','rs11865086','rs4792891'],
    'cM': [0,0,0,0,0],
    'pos': [58155355,81921705,175949971,30119172,45896132]
})

In [25]:
MAP = pd.concat([MAP, new_MAP_rows], ignore_index = True)

In [26]:
MAP.tail()

,chr,rsID,cM,pos
9989,2,rs848293,0,58155355
9990,3,rs10779987,0,81921705
9991,4,rs62340585,0,175949971
9992,16,rs11865086,0,30119172
9993,17,rs4792891,0,45896132


Now, recalculate the summary statistics. The results should include our very significant loci.

In [27]:
summary_stats = calculate_GWAS_summary_stats_from_PED(PED)

In [28]:
summary_stats[summary_stats['P_Value'] < 0.001]

,SNP,Allele_Frequency,Beta,Odds_Ratio,P_Value
469,rs1_470,0.28793,0.047525,1.048673,6.709569e-04
1169,rs2_349,0.45021,0.046613,1.047717,2.398810e-04
1836,rs3_217,0.42116,-0.048621,0.952542,1.413980e-04
2656,rs4_383,0.24153,0.048885,1.050099,9.759826e-04
2711,rs4_438,0.34079,-0.045140,0.955864,7.191784e-04
2979,rs5_79,0.12279,0.067004,1.069300,5.082493e-04
3147,rs5_247,0.47287,0.044800,1.045818,4.032052e-04
3200,rs5_300,0.26862,0.052631,1.054041,2.295116e-04
3577,rs6_79,0.29646,0.051740,1.053102,1.863902e-04
6828,rs12_422,0.13725,-0.061698,0.940166,7.950125e-04


Save the dataframes.

In [29]:
MAP.to_csv('GWAS_tutorial.map', sep = '\t', header = True, index = False)

In [30]:
PED.to_csv('GWAS_tutorial.ped', sep = '\t', header = True, index = False)